In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:\\Windows\\Fonts\\malgun.ttf").get_name()
plt.rc("font", family = font_name)

import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"] = False

In [2]:
# import folium
# import json

# 1. 데이터 로드 및 전처리

- 지점별일자별 교통량(TOPIS) 
- 2018-2022년 월간데이터 평균
- https://topis.seoul.go.kr/refRoom/openRefRoom_2.do

## (1) 데이터 로드 

In [3]:
date =[]
for y in range(18,23) :
    for m in range(1,13) :
        date.append("20{}년 {:02d}월".format(y, m))

In [4]:
name=[]
for n in range(len(date)) :
     name.append("f{}".format(n))  

In [5]:
dfs=[]
for i in range(len(name)) :
    name[i]=pd.read_excel("../../Data/교통량\\{} 서울시 교통량 조사자료_인코딩.xlsx".format(date[i]),
            sheet_name='{}'.format(date[i]))
    dfs.append(name[i])

In [6]:
print(dfs)

[            일자 요일        지점명  지점번호  방향            구분      0시      1시      2시  \
0     20180101  월  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역   657.0   617.0   405.0   
1     20180102  화  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역   382.0   256.0   166.0   
2     20180103  수  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역   641.0   492.0   310.0   
3     20180104  목  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역   661.0   483.0   357.0   
4     20180105  금  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역   764.0   524.0   361.0   
...        ... ..        ...   ...  ..           ...     ...     ...     ...   
8365  20180127  토     서부간선도로  F-09  유출      목동교->신정교  2556.0  2077.0  1489.0   
8366  20180128  일     서부간선도로  F-09  유출      목동교->신정교  2319.0  1767.0  1263.0   
8367  20180129  월     서부간선도로  F-09  유출      목동교->신정교  1516.0   860.0   638.0   
8368  20180130  화     서부간선도로  F-09  유출      목동교->신정교  2000.0  1240.0   826.0   
8369  20180131  수     서부간선도로  F-09  유출      목동교->신정교  1827.0  1273.0   900.0   

          3시  ...     14시     15시     

## (2) 5년치 데이터 concat

In [7]:
traffic=pd.concat(dfs)
traffic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495940 entries, 0 to 8617
Data columns (total 34 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   일자           495940 non-null  int64  
 1   요일           495940 non-null  object 
 2   지점명          495940 non-null  object 
 3   지점번호         495940 non-null  object 
 4   방향           495940 non-null  object 
 5   구분           495940 non-null  object 
 6   0시           442513 non-null  float64
 7   1시           442669 non-null  float64
 8   2시           442735 non-null  float64
 9   3시           442686 non-null  float64
 10  4시           443122 non-null  float64
 11  5시           443369 non-null  float64
 12  6시           443365 non-null  float64
 13  7시           443383 non-null  float64
 14  8시           443294 non-null  float64
 15  9시           443348 non-null  float64
 16  10시          443306 non-null  float64
 17  11시          443290 non-null  float64
 18  12시          443385 non-nu

# 2. 지점별 교통량

## (1) 결측치 확인 및 전처리 등

In [8]:
traffic.isna().sum(axis=0)

일자                  0
요일                  0
지점명                 0
지점번호                0
방향                  0
구분                  0
0시              53427
1시              53271
2시              53205
3시              53254
4시              52818
5시              52571
6시              52575
7시              52557
8시              52646
9시              52592
10시             52634
11시             52650
12시             52555
13시             52692
14시             52404
15시             52238
16시             52070
17시             52052
18시             51898
19시             52117
20시             51930
21시             52000
22시             53415
23시             54051
월              487570
요일(2)          471370
Unnamed: 30    495940
Unnamed: 31    495939
dtype: int64

In [9]:
traffic[traffic["요일(2)"].isna() == False].head()   # 요일(2)는 평일 주말로 삭제가능
traffic["요일(2)"].unique()

array([nan, '주말', '평일'], dtype=object)

In [10]:
traffic["Unnamed: 30"].unique()            # 두 컬럼 모두 삭제 가능
traffic["Unnamed: 31"].unique()

array([nan, '주) 서부간선도로(지상)은 서부간선지하도로 개통된 9월1일에 자동차전용도로에서 해제됨'],
      dtype=object)

In [11]:
traffic[traffic["월"].isna() == False]  # 삭제
traffic["월"].unique()

array([nan,  1.])

In [12]:
traffic["요일"].unique()

array(['월', '화', '수', '목', '금', '토', '일'], dtype=object)

In [13]:
traffic["지점번호"].unique()

array(['A-01', 'A-02', 'A-03', 'A-04', 'A-05', 'A-06', 'A-07', 'A-08',
       'A-09', 'A-10', 'A-11', 'A-12', 'A-13', 'A-14', 'A-15', 'A-16',
       'A-17', 'A-18', 'A-19', 'A-20', 'A-21', 'A-22', 'A-23', 'A-24',
       'B-01', 'B-02', 'B-03', 'B-04', 'B-05', 'B-06', 'B-07', 'B-08',
       'B-09', 'B-10', 'B-11', 'B-12', 'B-13', 'B-14', 'B-15', 'B-16',
       'B-17', 'B-18', 'B-19', 'B-20', 'B-21', 'B-22', 'B-23', 'B-24',
       'B-25', 'B-26', 'B-27', 'B-28', 'B-29', 'B-30', 'B-31', 'B-32',
       'B-33', 'B-34', 'B-35', 'B-36', 'B-37', 'C-01', 'C-03', 'C-04',
       'C-05', 'C-06', 'C-07', 'C-08', 'C-09', 'C-10', 'C-11', 'C-12',
       'C-13', 'C-14', 'C-15', 'C-16', 'C-17', 'C-18', 'C-19', 'C-20',
       'C-21', 'D-01', 'D-02', 'D-03', 'D-04', 'D-05', 'D-06', 'D-07',
       'D-08', 'D-09', 'D-10', 'D-11', 'D-12', 'D-13', 'D-14', 'D-15',
       'D-16', 'D-17', 'D-18', 'D-19', 'D-20', 'D-21', 'D-22', 'D-23',
       'D-24', 'D-25', 'D-26', 'D-27', 'D-28', 'D-29', 'D-30', 'D-31',
      

In [14]:
traffic["지점명"].unique()

array(['성산로(금화터널)', '사직로(사직터널)', '자하문로(자하문터널)', '대사관로(삼청터널)', '율곡로(안국역)',
       '창경궁로(서울여자대학교)', '대학로(한국방송통신대학교)', '종로(동묘앞역)', '퇴계로(신당역)',
       '동호로(장충체육관)', '장충단로(장충단공원)', '퇴계로(회현역)', '세종대로(서울역)',
       '새문안로(서울역사박물관)', '종로(종로3가역)', '서소문로(시청역)', '세종대로(시청역2)',
       '을지로(을지로3가역)', '칠패로(숭례문)', '남산1호터널', '남산2호터널', '남산3호터널',
       '소월로(회현역)', '소파로(숭의여자대학교)', '도봉로(도봉산역)', '동일로(의정부IC)', '아차산로(워커힐)',
       '망우로(망우리공원)', '경춘북로(중랑경찰서)', '화랑로(조선왕릉)', '북부간선도로(신내IC)',
       '서하남로(서하남IC)', '천호대로(상일IC)', '올림픽대로(강일IC)', '경부고속도로(양재IC)',
       '송파대로(복정역)', '밤고개로(세곡동사거리)', '분당수서로(성남시계)', '과천대로(남태령역)',
       '양재대로(양재IC)', '반포대로(우면산터널)', '시흥대로(석수역)', '금오로(광명시계)', '오리로(광명시계)',
       '개봉로(개봉교)', '광명대교(광명시계)', '철산교(광명시계)', '금천교(광명시계)', '금하로(광명시계)',
       '오정로(부천시계)', '화곡로(화곡로입구)', '경인고속국도(신월IC)', '경인로(유한공고)',
       '신정로(작동터널)', '김포대로(개화교)', '올림픽대로(개화IC)', '통일로(고양시계)', '서오릉로(고양시계)',
       '수색로(고양시계)', '강변북로(난지한강공원)', '강변북로(구리시계)', '행주대교', '가양대교', '성산대교',
       '양화대교', '서강대교', '마포대교', '원효대교', '

In [15]:
traffic["방향"].unique()

array(['유입', '유출', '봉원고가차도->독립문역', '독립문역->봉원고가차도', '독립문역->사직단',
       '사직단->독립문역', '석파정->청운초등학교', '청운초등학교->석파정', '삼청각->삼청공원입구',
       '삼청공원입구->삼청각', '안국역->안국동사거리', '안국동사거리->안국역', '성균관대입구->원남동사거리',
       '원남동사거리->성균관대입구', '혜화동로터리->이화사거리', '이화사거리->혜화동로터리', '신설동역->동묘앞역',
       '동묘앞역->신설동역', '상왕십리역->신당역', '신당역->상왕십리역', '약수역->장충체육관',
       '장충체육관->약수역', '남산2호터널북단->장충체육관', '장충체육관->남산2호터널북단', '서울역->회현역',
       '회현역->서울역', '서울역->숭례문', '숭례문->서울역', '서울역사박물관->광화문역',
       '광화문역->서울역사박물관', '종로3가->종로2가', '종로2가->종로3가', '경찰청앞->시청역',
       '시청역->경찰청앞', '서울시의회->덕수궁', '덕수궁->서울시의회', '을지로3가역->을지로2가',
       '을지로2가->을지로3가역', '염천교->숭례문', '숭례문->염천교', '한남1고가차도->남산1호터널북단',
       '남산1호터널북단->한남1고가차도', '용암초등학교->남산2호터널북단', '남산2호터널북단->용암초등학교',
       '용암초등학교->남산3호터널북단', '남산3호터널북단->용암초등학교', '도동삼거리->숭례문', '숭례문->도동삼거리',
       '백범광장->숭의여대', '숭의여대->백범광장', '도봉로의정부시계->도봉산역', '도봉산역->도봉로의정부시계',
       '의정부IC->수락지하차도', '수락지하차도->의정부IC', '아차산로구리시계->광나루역',
       '광나루역->아차산로구리시계', '망우로구리시계->동부제일병원', '동부제일병원->망우로구리시계',

In [16]:
traffic.loc[(traffic["방향"] != "유출") & (traffic["방향"] != "유입"),"dummy"]=traffic.loc[(traffic["방향"] != "유출") & (traffic["방향"] != "유입"),"방향"]

In [17]:
traffic.loc[(traffic["방향"] != "유출") & (traffic["방향"] != "유입"), "방향"]=traffic.loc[(traffic["방향"] != "유출") & (traffic["방향"] != "유입"),"구분"]

In [18]:
traffic.loc[(traffic["구분"] == "유출") | (traffic["구분"] == "유입"), "구분"]=traffic.loc[(traffic["구분"] == "유출") | (traffic["구분"] == "유입"), "dummy"]

In [19]:
traffic["방향"].unique()

array(['유입', '유출'], dtype=object)

In [20]:
hour=[(str(i)+"시") for i in range(0,24)]
for i in range(0,24) :
    hour[i]
    print(hour[i], traffic[hour[i]][traffic[hour[i]].isna()].unique())

0시 [nan]
1시 [nan]
2시 [nan]
3시 [nan]
4시 [nan]
5시 [nan]
6시 [nan]
7시 [nan]
8시 [nan]
9시 [nan]
10시 [nan]
11시 [nan]
12시 [nan]
13시 [nan]
14시 [nan]
15시 [nan]
16시 [nan]
17시 [nan]
18시 [nan]
19시 [nan]
20시 [nan]
21시 [nan]
22시 [nan]
23시 [nan]


In [21]:
del traffic["요일(2)"] 
del traffic["월"] 
del traffic["Unnamed: 30"] 
del traffic["Unnamed: 31"] 
del traffic['dummy']

In [22]:
traffic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495940 entries, 0 to 8617
Data columns (total 30 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   일자      495940 non-null  int64  
 1   요일      495940 non-null  object 
 2   지점명     495940 non-null  object 
 3   지점번호    495940 non-null  object 
 4   방향      495940 non-null  object 
 5   구분      495940 non-null  object 
 6   0시      442513 non-null  float64
 7   1시      442669 non-null  float64
 8   2시      442735 non-null  float64
 9   3시      442686 non-null  float64
 10  4시      443122 non-null  float64
 11  5시      443369 non-null  float64
 12  6시      443365 non-null  float64
 13  7시      443383 non-null  float64
 14  8시      443294 non-null  float64
 15  9시      443348 non-null  float64
 16  10시     443306 non-null  float64
 17  11시     443290 non-null  float64
 18  12시     443385 non-null  float64
 19  13시     443248 non-null  float64
 20  14시     443536 non-null  float64
 21  15시     4437

## (2) 결측치는 삭제(주로 빈칸)

In [23]:
traffic=traffic.dropna(axis=0)

In [24]:
traffic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 432610 entries, 0 to 8617
Data columns (total 30 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   일자      432610 non-null  int64  
 1   요일      432610 non-null  object 
 2   지점명     432610 non-null  object 
 3   지점번호    432610 non-null  object 
 4   방향      432610 non-null  object 
 5   구분      432610 non-null  object 
 6   0시      432610 non-null  float64
 7   1시      432610 non-null  float64
 8   2시      432610 non-null  float64
 9   3시      432610 non-null  float64
 10  4시      432610 non-null  float64
 11  5시      432610 non-null  float64
 12  6시      432610 non-null  float64
 13  7시      432610 non-null  float64
 14  8시      432610 non-null  float64
 15  9시      432610 non-null  float64
 16  10시     432610 non-null  float64
 17  11시     432610 non-null  float64
 18  12시     432610 non-null  float64
 19  13시     432610 non-null  float64
 20  14시     432610 non-null  float64
 21  15시     4326

## (3) 시간별 평균 교통량 산출

In [25]:
hour

['0시',
 '1시',
 '2시',
 '3시',
 '4시',
 '5시',
 '6시',
 '7시',
 '8시',
 '9시',
 '10시',
 '11시',
 '12시',
 '13시',
 '14시',
 '15시',
 '16시',
 '17시',
 '18시',
 '19시',
 '20시',
 '21시',
 '22시',
 '23시']

In [26]:
traffic["시간대별평균"]=traffic[hour].mean(axis=1)
traffic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 432610 entries, 0 to 8617
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   일자      432610 non-null  int64  
 1   요일      432610 non-null  object 
 2   지점명     432610 non-null  object 
 3   지점번호    432610 non-null  object 
 4   방향      432610 non-null  object 
 5   구분      432610 non-null  object 
 6   0시      432610 non-null  float64
 7   1시      432610 non-null  float64
 8   2시      432610 non-null  float64
 9   3시      432610 non-null  float64
 10  4시      432610 non-null  float64
 11  5시      432610 non-null  float64
 12  6시      432610 non-null  float64
 13  7시      432610 non-null  float64
 14  8시      432610 non-null  float64
 15  9시      432610 non-null  float64
 16  10시     432610 non-null  float64
 17  11시     432610 non-null  float64
 18  12시     432610 non-null  float64
 19  13시     432610 non-null  float64
 20  14시     432610 non-null  float64
 21  15시     4326

In [27]:
traffic.head()

,일자,요일,지점명,지점번호,방향,구분,0시,1시,2시,3시,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,시간대별평균
0,20180101,월,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,657.0,617.0,405.0,279.0,...,1592.0,1515.0,1447.0,1192.0,1037.0,918.0,881.0,819.0,533.0,930.125000
1,20180102,화,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,382.0,256.0,166.0,171.0,...,1835.0,1875.0,2021.0,2015.0,1612.0,1236.0,1308.0,1209.0,951.0,1402.458333
2,20180103,수,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,641.0,492.0,310.0,238.0,...,2007.0,1940.0,2096.0,2209.0,1653.0,1387.0,1295.0,1248.0,956.0,1478.333333
3,20180104,목,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,661.0,483.0,357.0,279.0,...,2030.0,2058.0,2123.0,2113.0,1619.0,1327.0,1379.0,1318.0,984.0,1489.333333
4,20180105,금,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,764.0,524.0,361.0,282.0,...,2134.0,2163.0,2282.0,2307.0,1723.0,1577.0,1460.0,1371.0,1102.0,1561.583333


In [28]:
traffic.to_csv("../../Data/회귀분석/교통량삭제.csv",encoding="utf-8")

## (4) 연도 산출

In [29]:
traffic["연도"]=traffic["일자"].astype(str).str.extract("^(20\d{2})")

In [30]:
traffic.head()

,일자,요일,지점명,지점번호,방향,구분,0시,1시,2시,3시,...,16시,17시,18시,19시,20시,21시,22시,23시,시간대별평균,연도
0,20180101,월,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,657.0,617.0,405.0,279.0,...,1515.0,1447.0,1192.0,1037.0,918.0,881.0,819.0,533.0,930.125000,2018
1,20180102,화,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,382.0,256.0,166.0,171.0,...,1875.0,2021.0,2015.0,1612.0,1236.0,1308.0,1209.0,951.0,1402.458333,2018
2,20180103,수,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,641.0,492.0,310.0,238.0,...,1940.0,2096.0,2209.0,1653.0,1387.0,1295.0,1248.0,956.0,1478.333333,2018
3,20180104,목,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,661.0,483.0,357.0,279.0,...,2058.0,2123.0,2113.0,1619.0,1327.0,1379.0,1318.0,984.0,1489.333333,2018
4,20180105,금,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,764.0,524.0,361.0,282.0,...,2163.0,2282.0,2307.0,1723.0,1577.0,1460.0,1371.0,1102.0,1561.583333,2018


## (5) 데이터프레임 정리

In [31]:
traffic2 = traffic[["연도","일자","지점번호","지점명","방향","구분","시간대별평균"]]

## (6) 연도별, 지점번호별 교통량평균

In [32]:
traffic3=traffic2.pivot_table(values="시간대별평균",index=["연도", "지점번호"],aggfunc="mean").reset_index()

## (7) 지점위치 정보

In [33]:
spot=pd.read_excel("../../Data/교통량\\2022년 12월 서울시 교통량 조사자료_인코딩.xlsx", sheet_name="수집지점 주소 및 좌표")

In [34]:
spot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   지점번호    141 non-null    object 
 1   지점명칭    139 non-null    object 
 2   검지기 유형  139 non-null    object 
 3   위도      139 non-null    float64
 4   경도      139 non-null    float64
 5   주소      139 non-null    object 
 6   도로명 주소  33 non-null     object 
 7   유입 방향   139 non-null    object 
 8   유출방향    139 non-null    object 
dtypes: float64(2), object(7)
memory usage: 10.2+ KB


In [35]:
spot=spot.loc[spot["주소"].isna()==False]
spot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139 entries, 0 to 138
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   지점번호    139 non-null    object 
 1   지점명칭    139 non-null    object 
 2   검지기 유형  139 non-null    object 
 3   위도      139 non-null    float64
 4   경도      139 non-null    float64
 5   주소      139 non-null    object 
 6   도로명 주소  33 non-null     object 
 7   유입 방향   139 non-null    object 
 8   유출방향    139 non-null    object 
dtypes: float64(2), object(7)
memory usage: 10.9+ KB


In [36]:
spot["지점번호"].unique()

array(['A-01', 'A-02', 'A-03', 'A-04', 'A-05', 'A-06', 'A-07', 'A-08',
       'A-09', 'A-10', 'A-11', 'A-12', 'A-13', 'A-14', 'A-15', 'A-16',
       'A-17', 'A-18', 'A-19', 'A-20', 'A-21', 'A-22', 'A-23', 'A-24',
       'B-01', 'B-02', 'B-03', 'B-04', 'B-05', 'B-06', 'B-07', 'B-08',
       'B-09', 'B-10', 'B-11', 'B-12', 'B-13', 'B-14', 'B-15', 'B-16',
       'B-17', 'B-18', 'B-19', 'B-20', 'B-21', 'B-22', 'B-23', 'B-24',
       'B-25', 'B-26', 'B-27', 'B-28', 'B-29', 'B-30', 'B-31', 'B-32',
       'B-33', 'B-34', 'B-35', 'B-36', 'B-37', 'B-38', 'C-01', 'C-02',
       'C-03', 'C-04', 'C-05', 'C-06', 'C-07', 'C-08', 'C-09', 'C-10',
       'C-11', 'C-12', 'C-13', 'C-14', 'C-15', 'C-16', 'C-17', 'C-18',
       'C-19', 'C-20', 'C-21', 'D-01', 'D-02', 'D-03', 'D-04', 'D-05',
       'D-06', 'D-07', 'D-08', 'D-09', 'D-10', 'D-11', 'D-12', 'D-13',
       'D-14', 'D-15', 'D-16', 'D-17', 'D-18', 'D-19', 'D-20', 'D-21',
       'D-22', 'D-23', 'D-24', 'D-25', 'D-26', 'D-27', 'D-28', 'D-29',
      

In [37]:
spot2=spot[["지점번호","위도","경도","주소"]]

## (8) 데이터 프레임 머지

In [38]:
df=pd.merge(traffic3, spot2, on="지점번호", how="outer")

In [39]:
df

,연도,지점번호,시간대별평균,위도,경도,주소
0,2018,A-01,1380.083877,37.568588,126.948436,서울시 서대문구 신촌동 1-142
1,2019,A-01,1371.932479,37.568588,126.948436,서울시 서대문구 신촌동 1-142
2,2020,A-01,1228.536487,37.568588,126.948436,서울시 서대문구 신촌동 1-142
3,2021,A-01,1178.533722,37.568588,126.948436,서울시 서대문구 신촌동 1-142
4,2022,A-01,1232.255883,37.568588,126.948436,서울시 서대문구 신촌동 1-142
...,...,...,...,...,...,...
674,2022,F-09,975.891096,37.468942,126.883673,서울시 금천구 가산동 338-10
675,2022,B-38,1431.364265,37.683253,127.052529,서울시 노원구 상계동 1180-13
676,2022,C-02,1061.488243,37.556598,126.885535,서울시 마포구 상암동 1541
677,2022,D-46,2175.355525,37.520965,126.881826,서울시 영등포구 문래동6가 52


## (9) 서울시 추출

In [40]:
sido_name=[]
for a in df["주소"] :
    sido=a.split(" ")[0]
    if sido == "서울" :
        sido_name.append(sido+"시")
    elif sido == "서울시," :
        sido_name.append(sido[0:3])
    elif sido == "경기" :
        sido_name.append(sido+"도")
    else : 
        sido_name.append(sido)

In [41]:
df["시도"]=sido_name

In [42]:
df2=df[df["시도"]=="서울시"]

## (10) 자치구, 법정동 추출

In [43]:
gu_name=[]

for a in df2.주소 :
    gu_name.append(a.split(" ")[1])

In [44]:
df2["구별"]=gu_name

C:\Users\acorn\AppData\Local\Temp\ipykernel_1632\768043827.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["구별"]=gu_name


In [45]:
import requests

head = {"Authorization":"KakaoAK cbe1a3567ce9912ad46a4cf87043ad88"}

ka_dong1 = []
# ka_dong2 = []

for a in df2["주소"]:
    url = "https://dapi.kakao.com/v2/local/search/address.json?query={}".format(a)
    try :
        place1=requests.get(url,headers=head).json()['documents'][0]['address']['region_3depth_h_name']
#         place2=requests.get(url,headers=head).json()['documents'][0]['address']['region_3depth_name']
        ka_dong1.append(place1)
#         ka_dong2.append(place2)
    except :
        ka_dong1.append(a.split(" ")[2])
#         ka_dong2.append(a.split(" ")[2])

In [46]:
print(set(ka_dong1))

{'자양동', '한남동', '월계3동', '종로1.2.3.4가동', '신내1동', '약수동', '사당2동', '당산2동', '도곡2동', '서초3동', '반포2동', '장위2동', '반포3동', '이화동', '공항동', '독산1동', '소공동', '행운동', '신정4동', '월곡2동', '교남동', '명동', '고덕2동', '상계1동', '송파2동', '신월4동', '방배2동', '잠실7동', '방배본동', '삼성1동', '노량진1동', '영등포동', '신월7동', '망원1동', '장안1동', '천호2동', '사직동', '개포3동', '황학동', '논현1동', '홍은2동', '개봉1동', '혜화동', '광장동', '수색동', '반포4동', '신사동', '돈암1동', '상암동', '정릉3동', '장충동', '서초동', '역삼1동', '방배3동', '합정동', '신림동', '세곡동', '성북동', '창2동', '오류2동', '도봉1동', '증산동', '공릉2동', '숭인2동', '문래동', '여의동', '용답동', '일원1동', '대림2동', '청담동', '발산1동', '문정2동', '상계6.7동', '방화2동', '평창동', '시흥2동', '연희동', '이촌1동', '상일2동', '가산동', '신촌동', '화곡8동', '용산2가동', '풍납1동', '시흥3동', '청운효자동', '양평1동', '궁동', '신월3동', '옥수동', '잠실6동', '이태원2동', '신월5동', '신도림동', '양재2동', '개봉3동', '회현동', '구산동', '송정동'}


In [47]:
# print(set(ka_dong2))

In [48]:
df2["동별"]=ka_dong1
# df2["행정동2"]=ka_dong2

C:\Users\acorn\AppData\Local\Temp\ipykernel_1632\3911538671.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["동별"]=ka_dong1


In [49]:
df2.head()

,연도,지점번호,시간대별평균,위도,경도,주소,시도,구별,동별
0,2018,A-01,1380.083877,37.568588,126.948436,서울시 서대문구 신촌동 1-142,서울시,서대문구,신촌동
1,2019,A-01,1371.932479,37.568588,126.948436,서울시 서대문구 신촌동 1-142,서울시,서대문구,신촌동
2,2020,A-01,1228.536487,37.568588,126.948436,서울시 서대문구 신촌동 1-142,서울시,서대문구,신촌동
3,2021,A-01,1178.533722,37.568588,126.948436,서울시 서대문구 신촌동 1-142,서울시,서대문구,신촌동
4,2022,A-01,1232.255883,37.568588,126.948436,서울시 서대문구 신촌동 1-142,서울시,서대문구,신촌동


In [50]:
# del df2["시도"]

In [51]:
df2

,연도,지점번호,시간대별평균,위도,경도,주소,시도,구별,동별
0,2018,A-01,1380.083877,37.568588,126.948436,서울시 서대문구 신촌동 1-142,서울시,서대문구,신촌동
1,2019,A-01,1371.932479,37.568588,126.948436,서울시 서대문구 신촌동 1-142,서울시,서대문구,신촌동
2,2020,A-01,1228.536487,37.568588,126.948436,서울시 서대문구 신촌동 1-142,서울시,서대문구,신촌동
3,2021,A-01,1178.533722,37.568588,126.948436,서울시 서대문구 신촌동 1-142,서울시,서대문구,신촌동
4,2022,A-01,1232.255883,37.568588,126.948436,서울시 서대문구 신촌동 1-142,서울시,서대문구,신촌동
...,...,...,...,...,...,...,...,...,...
674,2022,F-09,975.891096,37.468942,126.883673,서울시 금천구 가산동 338-10,서울시,금천구,가산동
675,2022,B-38,1431.364265,37.683253,127.052529,서울시 노원구 상계동 1180-13,서울시,노원구,상계1동
676,2022,C-02,1061.488243,37.556598,126.885535,서울시 마포구 상암동 1541,서울시,마포구,상암동
677,2022,D-46,2175.355525,37.520965,126.881826,서울시 영등포구 문래동6가 52,서울시,영등포구,문래동


## (11) 연도별, 구별, 행정동별 교통량 평균

In [52]:
df3=df2.groupby(["구별","동별","연도"])[["시간대별평균"]].mean()

In [53]:
df3

시간대별평균
구별  동별   연도               
강남구 개포3동 2018  2834.863043
         2019  2823.592606
         2020  2763.897405
         2021  2720.475320
         2022  2728.373885
...                    ...
중랑구 신내1동 2018  1831.318919
         2019  1781.473679
         2020  1673.872625
         2021  1675.710956
         2022  1693.454065

[487 rows x 1 columns]

### ※ 파일저장

In [54]:
df3.to_csv("../../Data/회귀분석/교통량_18_22.csv", 
          encoding="utf-8",header=False)

In [55]:
# 행정동이 같지 않을 때
# df2.loc[df2["행정동1"] != df2["행정동2"],["주소","구별","행정동1","행정동2"]]